### Test on ADVP 1

In [1]:
import pandas as pd
from gwas_table_extraction import *

/Users/justpqa/advpai/venv/lib/python3.10/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4
/Users/justpqa/advpai/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Import reference table

In [2]:
advp = pd.read_csv("advp.variant.records.hg38.tsv", sep='\t')
advp.head()

,#dbSNP_hg38_chr,dbSNP_hg38_position,Top SNP,P-value,LocusName,RA 1(Reported Allele 1),nonref_allele,nonref_effect,OR_nonref,nearest_gene_symb,Study type,Study Design,Pubmed PMID,Population_map,Cohort_simple3,Sample size,Analysis group,Phenotype,Phenotype-derived,most_severe_consequence
0,chr1,6434683,rs12074379,0.00726,ESPN,T,T,NR,NaN,ESPN,SNP-based,Disease risk,30636644,Caucasian,"ADGC, CHS, CHARGE, HRS",10191,Plan 3 (only females),AD,AD,intron_variant
1,chr1,6434683,rs12074379,8.51E-40,NR,T,T,NR,NaN,ESPN,SNP-based,eQTL,30636644,Caucasian,"ADGC, CHS, CHARGE, HRS",10191,Plan 3 (only females),ESPN (ILMN_1806710) expression,Expression,intron_variant
2,chr1,8708071,rs112053331,0.0009,RERE,NR,NR,NR,NaN,RERE,SNP-based,Cross phenotype,30010129,Caucasian,IGAP,54162,All,AD,AD,intron_variant
3,chr1,8708071,rs112053331,0.08392,NaN,NR,NR,NR,NaN,RERE,Gene-based,Cross phenotype,30010129,Caucasian,IGAP,54162,All,AD,AD,intron_variant
4,chr1,11487007,rs2379135,0.0156,PTCHD2,NR,NR,NR,NaN,DISP3,SNP-based,Endophenotype,22245343,Caucasian,ADNI,757,All,MRI,Imaging,intron_variant


In [3]:
advp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6346 entries, 0 to 6345
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   #dbSNP_hg38_chr          6346 non-null   object 
 1   dbSNP_hg38_position      6346 non-null   int64  
 2   Top SNP                  6346 non-null   object 
 3   P-value                  6338 non-null   object 
 4   LocusName                6244 non-null   object 
 5   RA 1(Reported Allele 1)  6222 non-null   object 
 6   nonref_allele            6222 non-null   object 
 7   nonref_effect            6224 non-null   object 
 8   OR_nonref                3390 non-null   float64
 9   nearest_gene_symb        6336 non-null   object 
 10  Study type               6346 non-null   object 
 11  Study Design             6346 non-null   object 
 12  Pubmed PMID              6346 non-null   int64  
 13  Population_map           6346 non-null   object 
 14  Cohort_simple3          

Test papers along with their PMC and PMID
- 1-s2.0-S0197458018303816-main.pdf - 30448613 - PMC6331247
- 1-s2.0-S0197458019300727-main.pdf - 30979435 - PMC6783343
- s11357-019-00071-5.pdf - 31055733 - PMC6544706
- s41588-018-0311-9.pdf - 30617256 - PMC6836675
- s41588-019-0358-2.pdf - 30820047 - PMC6463297

In [4]:
# paper file - PMID - PMCID - table id
test_papers_info = [
    ("test_papers/1-s2.0-S0197458018303816-main.pdf", 30448613, "PMC6331247", [1, 2, 3]),
    ("test_papers/1-s2.0-S0197458019300727-main.pdf", 30979435, "PMC6783343", [1]),
    ("test_papers/s11357-019-00071-5.pdf", 31055733, "PMC6544706", [1, 3, 4]), # got error in PMC
    ("test_papers/s41588-018-0311-9.pdf", 30617256, "PMC6836675", [1]),
    ("test_papers/s41588-019-0358-2.pdf", 30820047, "PMC6463297", [1, 2])
]

for file_name, pmid, pmcid, inx_lst in test_papers_info:
    try:
        df_lst = extract_tables_lst_from_paper(pmcid, file_name)
    except:
        print(f"Error while extracting tables from {file_name}")
    try:
        for i in inx_lst:
            df_lst[i-1].to_csv(f"test_tables/{file_name.split('/')[-1].replace('.pdf', '')}_table_{i}.csv", index=False)
    except:
        print(f"Not accurate extraction for {file_name}")

Successfully retrieve number of columns


objc[2241]: Class AVFFrameReceiver is implemented in both /Users/justpqa/advpai/venv/lib/python3.10/site-packages/av/.dylibs/libavdevice.62.1.100.dylib (0x136d043a8) and /Users/justpqa/advpai/venv/lib/python3.10/site-packages/cv2/.dylibs/libavdevice.61.3.100.dylib (0x13ebcc3a8). This may cause spurious casting failures and mysterious crashes. One of the duplicates must be removed or renamed.
objc[2241]: Class AVFAudioReceiver is implemented in both /Users/justpqa/advpai/venv/lib/python3.10/site-packages/av/.dylibs/libavdevice.62.1.100.dylib (0x136d043f8) and /Users/justpqa/advpai/venv/lib/python3.10/site-packages/cv2/.dylibs/libavdevice.61.3.100.dylib (0x13ebcc3f8). This may cause spurious casting failures and mysterious crashes. One of the duplicates must be removed or renamed.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `do

Not accurate extraction for test_papers/1-s2.0-S0197458018303816-main.pdf
Successfully retrieve number of columns


Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.


Error while extracting tables from test_papers/s11357-019-00071-5.pdf
Not accurate extraction for test_papers/s11357-019-00071-5.pdf
Successfully retrieve number of columns


Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.


Successfully retrieve number of columns


Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.


Not accurate extraction for test_papers/s41588-019-0358-2.pdf


In [ ]:
referencing_col_df = pd.read_csv("Rules for harmonizing ADVP papers - Main cols.csv")
referencing_col_df["column_with_context"] = referencing_col_df.apply(lambda x: x["column"] if pd.isna(x["description"]) else x["column"] + ": " + x["description"], axis = 1)

gwas_column_matching_engine = GWASColumnMatchingEngine(referencing_col_df)

for file in os.listdir("./test_tables"):
    if "table" in file and ".csv" in file and "harmonized" not in file:
        print(file)
        df = pd.read_csv(f"./test_tables/{file}")
        df.columns = ['' if 'Unnamed:' in col else col for col in df.columns]
        col_to_ref_col = gwas_column_matching_engine.match_many_col_to_ref_col(df)
        for ref_col in col_to_ref_col:
            print(f"{ref_col}: {col_to_ref_col[ref_col]}")
        print()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


s11357-019-00071-5_table_1.csv
Locus: [('Gene', 'Gene: CR1, BIN1, OR2S2, MS4A6A, MS4A4E, ', 0.5280643701553345)]
Chr: [('Chr', 'Chromosome number: 1, 2, 9, 11, 14, ', 0.6727339625358582)]
P-value: [('p value.', 'P-value value: 4.70 /C2 10 /C0 6, 3.67 3 10 L 8, 8.45 3 10 L 7, 8.68 /C2 10 /C0 6, 2.10 /C2 10 /C0 6, ', 0.4075296223163605), ('Age 60 e 79 y (younger).p value', 'P-value value: 5.41 /C2 10 /C0 9, 6.06 3 10 L 14, 5.43 3 10 L 6, 1.08 /C2 10 /C0 5, 1.45 /C2 10 /C0 6, ', 0.4264649748802185), ('Age /C21 80 y (older).p value', 'P-value value: 0.027, 0.340, 0.363, 3.38 /C2 10 /C0 4, 5.53 /C2 10 /C0 4, ', 0.5158348083496094), ('Heterogeneity between the younger & older age groups.p value', 'P-value value: 0.159, 6.86 3 10 L 4, 0.0695, 0.587, 0.863, ', 0.5418775677680969)]
SNP: [('Top SNP in gene.', 'top Single nucleotide polymorphism identifier in gene: rs3818361, rs6431219, rs12378268, rs1834550, rs184909761, ', 0.6773247122764587)]
RA: [('A1/A2.', 'effect allele / tested allele/Othe

In [ ]:
modified_df_all = None
for file in os.listdir("./test_tables"):
    if ".csv" in file and "table" in file and "harmonized" not in file:
        df = pd.read_csv(f"./test_tables/{file}")
        df.columns = ['' if 'Unnamed:' in col else col for col in df.columns]
        modified_df = format_original_table(df, gwas_column_matching_engine, remove_unique_col = False)
        modified_df["file_name"] = file
        if modified_df_all is None:
            modified_df_all = modified_df.copy()
        else:
            modified_df_all = pd.concat([modified_df_all, modified_df], ignore_index = True)
        modified_df.to_csv(f"./harmonized_test_tables/{file.replace('.csv', '')}_harmonized.csv", index = False)
modified_df_all.to_csv("./harmonized_test_tables/harmonized_table.csv", index = False)